<a href="https://colab.research.google.com/github/Loumstar/Improving-RoBERTa/blob/master/data_exploration_plus_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALLATIONS


All 

In [ ]:
!nvidia-smi

Tue Mar  1 19:25:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install tensorboardx
!pip install wandb

     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 6.5 MB 53.3 MB/s 
     |████████████████████████████████| 67 kB 7.2 MB/s 
     |████████████████████████████████| 895 kB 88.1 MB/s 
     |████████████████████████████████| 596 kB 87.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
     |████████████████████████████████| 125 kB 4.0 MB/s 
     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 181 kB 74.7 MB/s 
     |████████████████████████████████| 144 kB 58.7 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c1a49e55d058a6dfc0e7df9cb6336c1911bb44e01c97d344

In [ ]:
import torch
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer
from transformers import BertPreTrainedModel, BertModel

from transformers import DebertaV2Tokenizer, DebertaV2PreTrainedModel, DebertaV2Model

import pandas as pd
import numpy as np
import os

from sklearn.metrics import classification_report

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  DEVICE = 'cpu'
else:
  DEVICE = 'cuda:0'

In [ ]:
from urllib import request
import logging
from collections import Counter
from ast import literal_eval

In [ ]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/'
%mkdir './nlp-cswrk'
%cd './nlp-cswrk' 

Mounted at /content/drive
/content/drive/MyDrive
mkdir: cannot create directory ‘./nlp-cswrk’: File exists
/content/drive/MyDrive/nlp-cswrk


In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  b = f.read()
  outf.write(b.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv


In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  b = f.read()
  outf.write(b.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv


In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
from dont_patronize_me import DontPatronizeMe

In [ ]:
dpm = DontPatronizeMe('.', '.')
dpm.load_task1()

In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

# DATA EXPLORATION

## Imports

In [ ]:
import nltk

nltk.download("stopwords")
nltk.download("punkt")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import pandas as pd
import numpy as np


from tqdm import tqdm
from pprint import pprint
from operator import itemgetter
from matplotlib import pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#### Load Dataset

In [ ]:
fields = ["idx", "id", "category", "language", "text", "score"]

# with open("data/dontpatronizeme_pcl.tsv", "r") as f:
#     df = pd.read_csv(f, delimiter="\t", names=fields, header=None)

df = dpm.train_task1_df
df.dropna()

patronising_df = df.loc[df["label"] == 1]
unpatronising_df = df.loc[df["label"] == 0]

print(patronising_df.shape, unpatronising_df.shape)

NameError: ignored

## General Info

##### Balance of Classes

In [ ]:
figure, (binary_ax, multi_ax) = plt.subplots(1, 2, sharey=True)

figure.suptitle("Distribution of class labels")
figure.set_size_inches((12, 5))

df["orig_label"].map(lambda x: "Patronising" if x > 1 else "Unpatronising") \
    .hist(ax=binary_ax, bins=[-0.5, 0.5, 1.5], grid=False) 

df["orig_label"].hist(ax=multi_ax, bins=[-0.5, 0.5, 1.5, 2.5, 3.5, 4.5], grid=False) 

binary_ax.set_ylabel("Frequency")
binary_ax.set_xlabel("Label")
multi_ax.set_xlabel("Score")

##### Distribution of Languages

In [ ]:
df["country"].hist(figsize=(7, 5))

plt.title("Distribution of source-text languages")
plt.ylabel("Frequency")
plt.xlabel("Language")

#### Text Analysis

##### Average Character Length

In [ ]:
plt.title("Average Character Length of a sample")
plt.ylabel("Frequency")
plt.xlabel("Character Length")

# series = df["text"].str.len()

# print(series.max())
# series.hist(bins=200, figsize=(8, 6))

# unpatronising_df["text"].str.len().hist(bins=200, alpha=0.5, figsize=(8, 6))
# patronising_df["text"].str.len().hist(bins=200, alpha=0.5, figsize=(8, 6))

(unpatcounts, unpatbins) = np.histogram(unpatronising_df["text"].str.len(), bins=100, range=(0, 2000))
(patcounts, patbins) = np.histogram(patronising_df["text"].str.len(), bins=100, range=(0, 2000))

factor = 1/len(unpatronising_df["text"])
plt.hist(unpatbins[:-1], unpatbins, weights=factor*unpatcounts, alpha=0.6)

factor = 1/len(patronising_df["text"])
plt.hist(patbins[:-1], patbins, weights=factor*patcounts, alpha=0.6)

plt.xlim((0, 2000))

##### Average Number of Words

In [ ]:
def get_list_length(x):
    if type(x) != list:
        x = [str(x)]

    return len(x)

series = df["text"].str.split() \
    .map(get_list_length)

print(series.max())
series.hist(bins=200, figsize=(8, 6))

unpatronising_df["text"].str.split() \
    .map(get_list_length) \
    .hist(bins=200, alpha=0.5, figsize=(8, 6))

patronising_df["text"].str.split() \
    .map(get_list_length) \
    .hist(bins=200, alpha=0.5, figsize=(8, 6))

plot.title("Average Number of Words per sample")
plot.ylabel("Frequency")
plot.xlabel("Words")

plot.xlim((0, 300))

##### Average Word Length

In [ ]:
def mean_item_length(x):
    if type(x) != list:
        x = [str(x)]
    
    lengths = list(map(len, x))
    return np.mean(lengths)

series = df["text"].str.split() \
    .map(mean_item_length) \

print(series.max())
series.hist(bins=25, figsize=(8, 6))

unpatronising_df["text"].str.split() \
    .map(mean_item_length) \
    .hist(bins=25, alpha=0.5, figsize=(8, 6))

patronising_df["text"].str.split() \
    .map(mean_item_length) \
    .hist(bins=25, alpha=0.5, figsize=(8, 6))

plot.title("Average Word Length of a sample")
plot.ylabel("Frequency")
plot.xlabel("Word Length")

plot.xlim((0, 10))

#### $ n $-gram Analysis

In [ ]:
def is_not_stopword(x):
    return x not in stopwords.words()

def get_n_gram_frequency(n, sentences):
    n_grams = dict()

    for sentence in tqdm(sentences, unit="sentence"):
        if type(sentence) != str:
            continue

        words = list(filter(is_not_stopword, word_tokenize(sentence)))
    
        for i in range(len(words) + 1 - n):
            n_gram = " ".join(words[i:i+n])

            if n_gram in n_grams:
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    
    return list(zip(n_grams.keys(), n_grams.values()))


In [ ]:
n = 3

patr_n_gram = get_n_gram_frequency(n, patronising_df["text"])
unpatr_n_gram = get_n_gram_frequency(n, unpatronising_df["text"])

print(len(patr_n_gram), len(unpatr_n_gram))

common_patr_n_gram = sorted(patr_n_gram, key=itemgetter(1), 
                            reverse=True)[:50]

common_unpatr_n_gram = sorted(unpatr_n_gram, key=itemgetter(1), 
                              reverse=True)[:50]

common_patr_n_gram_words = set(map(itemgetter(0), common_patr_n_gram))
common_unpatr_n_gram_words = set(map(itemgetter(0), common_unpatr_n_gram))

common_n_grams = set.intersection(common_patr_n_gram_words, common_unpatr_n_gram_words)
print(f"Common {n}-grams between the patronising and unpatronising datasets.")
pprint(common_n_grams)

patr_difference_n_grams = common_patr_n_gram_words.difference(common_unpatr_n_gram_words)
print(f"{n}-grams in the patronising set but not in the unpatronising set.")
pprint(patr_difference_n_grams)

unpatr_difference_n_grams = common_unpatr_n_gram_words.difference(common_patr_n_gram_words)
print(f"{n}-grams in the unpatronising set but not in the patronising set.")
pprint(unpatr_difference_n_grams)

# MODEL

## DATA IMPORT

In [ ]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

In [ ]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

In [ ]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label
  })
trdf1 = pd.DataFrame(rows)

In [ ]:
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

trdf1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])

In [ ]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  intensity = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].orig_label.values[0]
  length = len(dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0])
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label,
      'keyword':keyword,
      'country':country,
      'intensity':intensity,
      'length':length
  })
tedf1 = pd.DataFrame(rows)

In [ ]:
def df_to_dict_of_lists(df):
    texts = df['text'].values.tolist()
    labels = df['label'].values.tolist()
    return {'texts':texts, 'labels':labels}

In [ ]:
class PatroniseDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, input_set):

        self.tokenizer = tokenizer
        self.texts = input_set['texts']
        self.labels = input_set['labels']
        
    def collate_fn(self, batch):

        texts = []
        labels = []

        for b in batch:
            texts.append(b['text'])
            labels.append(b['label'])

        #The maximum sequence size for BERT is 512 but here the tokenizer truncate sentences longer than 128 tokens.  
        # We also pad shorter sentences to a length of 128 tokens
        encodings = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=128)
        # labels = {}
        encodings['label'] =  torch.tensor(labels)
        
        return encodings
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
       
        item = {'text': self.texts[idx],
                'label': self.labels[idx]}
        return item

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v2-xlarge")

# we can check the parameters of this tokenizer
tokenizer

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/633 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PreTrainedTokenizer(name_or_path='microsoft/deberta-v2-xlarge', vocab_size=128000, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
trainset = df_to_dict_of_lists(trdf1)
testset = df_to_dict_of_lists(tedf1)

train_dataset = PatroniseDataset(tokenizer, trainset)
test_dataset = PatroniseDataset(tokenizer, testset)

## MODEL

In [ ]:
model = BertModel.from_pretrained("bert-base-cased")
# model = DebertaV2Model.from_pretrained("microsoft/deberta-v2-xlarge")

#180 M
print(f"Model size: {model.num_parameters()}")

In [ ]:
class BERT_patronise(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # BERT Model
        self.bert = BertModel(config)
        
        # Task A
        self.projection_a = torch.nn.Sequential(torch.nn.Dropout(0.2),
                                                torch.nn.Linear(config.hidden_size, 2))
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
 
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # Logits A
        logits_a = self.projection_a(outputs[1])
        
        return logits_a

In [ ]:
class DEBERTA_patronise(DebertaV2PreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # BERT Model
        self.deberta = DebertaV2Model(config)
        
        # Task A
        self.projection_a = torch.nn.Sequential(torch.nn.Dropout(0.2),
                                                torch.nn.Linear(config.hidden_size, 2))
        
        self.init_weights()

    def forward(
        self,
        input_ids = None,
        attention_mask = None,
        token_type_ids = None,
        position_ids = None,
        inputs_embeds = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None):
 
        outputs = self.deberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )

        # Logits A
        logits_a = self.projection_a(outputs[1])
        
        return logits_a

In [ ]:
class Trainer_patronise(Trainer):
    # def __init__(self, **kwargs):
    #     super().__init__(**kwargs)
    #     self.losses = []

    def compute_loss(self, model, inputs):
        labels = inputs.pop('label')

        outputs = model(**inputs)

        loss_task = nn.CrossEntropyLoss()

        loss = loss_task(outputs.view(-1, 2), labels)
        # self.losses.append(loss.item())
        
        return loss

In [ ]:
def main_patronise():

    #call our custom BERT model and pass as parameter the name of an available pretrained model
    model = BERT_patronise.from_pretrained("bert-base-cased")
    # model = DEBERTA_patronise.from_pretrained("microsoft/deberta-v2-xlarge")
    
    training_args = TrainingArguments(
        output_dir='./experiment/patronise',
        learning_rate = 0.0001,
        logging_steps= 100,
        per_device_train_batch_size=8,
        num_train_epochs = 3,
    )
    trainer = Trainer_patronise(
        model=model,                         
        args=training_args,                 
        train_dataset=train_dataset,                   
        data_collator=train_dataset.collate_fn
    )
    trainer.train()

    trainer.save_model('./models/ht_bert_finetuned/')


In [ ]:
main_patronise()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

RuntimeError: ignored

## EVALUATION

In [ ]:
def predict_patronise(input, tokenizer, model): 
  model.eval()
  encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=128)
  
  output = model(**encodings)
  preds = torch.max(output, 1)

  return {'prediction':preds[1], 'confidence':preds[0]}

In [ ]:
def evaluate(model, tokenizer, data_loader):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  with torch.no_grad():
    for data in tqdm(data_loader): 

      labels = {}
      labels['label'] = data['label']

      tweets = data['text']

      pred = predict_patronise(tweets, tokenizer, model)

      preds.append(pred['prediction'].tolist())
      tot_labels.append(labels['label'].tolist())

  # with the saved predictions and labels we can compute accuracy, precision, recall and f1-score
  report = classification_report(tot_labels, preds, target_names=[0,1], output_dict= True)

  return report

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

#your saved model name here
model_name = './models/ht_bert_finetuned/' 
model = BERT_patronise.from_pretrained(model_name)

# we don't batch our test set unless it's too big
test_loader = DataLoader(test_dataset)

report = evaluate(model, tokenizer, test_loader)

print(report)

print(report['accuracy'])
print(report[0]['f1-score'])
print(report[1]['f1-score'])

In [ ]:
print(report['accuracy'])
print(report[0]['f1-score'])
print(report[1]['f1-score'])

## PREPROCESSING
